In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
pwd

'C:\\Users\\CLIENTE'

In [3]:
cd "E:\DATASETS\Anime Database\"

E:\DATASETS\Anime Database


In [4]:
df = pd.read_csv("Anime.csv")

In [5]:
pd.set_option('display.max_columns', 50)
df.head(2)

,ID,Title,Synonyms,Japanese,English,Synopsis,Type,Episodes,Status,Start_Aired,End_Aired,Premiered,Broadcast,Producers,Licensors,Studios,Source,Genres,Themes,Demographics,Duration_Minutes,Rating,Score,Scored_Users,Ranked,Popularity,Members,Favorites
0,16498,Shingeki no Kyojin,"AoT, SnK",進撃の巨人,Attack on Titan,"Centuries ago, mankind was slaughtered to near...",TV,25.0,Finished Airing,"Apr 7, 2013","Sep 29, 2013",Spring 2013,Sundays at 0158 (JST),"Production I.G, Dentsu, Mainichi Broadcasting ...",Funimation,Wit Studio,Manga,"Action, Drama","Gore, Military, Survival",Shounen,24.0,R - 17+ (violence & profanity),8.531,519803.0,1002.0,1,3524109,155695
1,1535,Death Note,DN,デスノート,Death Note,"Brutal murders, petty thefts, and senseless vi...",TV,37.0,Finished Airing,"Oct 4, 2006","Jun 27, 2007",Fall 2006,Wednesdays at 0056 (JST),"VAP, Konami, Ashi Productions, Nippon Televisi...",VIZ Media,Madhouse,Manga,"Supernatural, Suspense",Psychological,Shounen,23.0,R - 17+ (violence & profanity),8.621,485487.0,732.0,2,3504535,159701


In [6]:
df.shape

(21460, 28)

##### Counting Nan values

In [7]:
df.isna().sum()
## MV == Missing Values
## Epsiodes the MV can be replaced by the average of episodes, which is 12 by season
## Duration_minutes he MV can be replaced by the average Duration which is 23 minutes
## Unfortunatebly score have many MV, so is possible that those Animes weren't popular, but getting rid of
## so many unpopular animes can baised our dataset, hence our model. But scoring them with the average can also be a bad idea
## and once again it can bias our model. Too much data is missing in that particular column, probably the best soulition is 
## just to drop it from the dataset. Same with Scored_users column
## Ranked, we can rank and clip the MV as the worst ranked animes.

ID                     0
Title                  0
Synonyms               0
Japanese               0
English                0
Synopsis               0
Type                   0
Episodes             547
Status                 0
Start_Aired            0
End_Aired              0
Premiered              0
Broadcast              0
Producers              0
Licensors              0
Studios                0
Source                 0
Genres                 0
Themes                 0
Demographics           0
Duration_Minutes     599
Rating                 0
Score               6898
Scored_Users        6898
Ranked              1924
Popularity             0
Members                0
Favorites              0
dtype: int64

In [8]:
df.Title[df['Score'].isna()]
## So seeing this I realized that the animes with MV on the score column are Animes that haven't been aired yet,
## or havne't finished yet. 
### But they've been announced or are being aired but aren't finished
#### So is safe and fair to 

362                                    Chainsaw Man
813                              Mob Psycho 100 III
833            Tate no Yuusha no Nariagari Season 3
890                             Spy x Family Part 2
915                Boku no Hero Academia 6th Season
                            ...                    
21455      Xiao Gongzhu Ai Wei La Yu Shenmi Wangguo
21456    Xiao Gongzhu Ai Wei La Yu Shenmi Wangguo 2
21457                       Yongzhe Chuang Mo Cheng
21458               Yu Shen Chuan: Wangzhe Shaonian
21459                                 Konglong Wang
Name: Title, Length: 6898, dtype: object

In [9]:
#  list(df.Title[df['Score'].isna()])
## I covnert the series to list to see all the data
### I can confirm that those animes are being aired but not finished or haven't even been aired yet just announced

In [10]:
## df['Episodes'] = df['Episodes'].fillna(value="ZZZZZ")
## df.loc[df.Episodes == "ZZZZZ"]
## df.loc[df.Episodes == "ZZZZZ" ].tail(30)
### Nan values on Episodes is because they are currently airing or not aired yet
#### Be careful because Boruto and One Piece, very popular animes have Nan on Episode Col.
## You can keep the Nan and work with them, after all Episodes is not part of the Features to run the model

In [11]:
## So is safe to drop those Nan's (Score, Episodes)

df = df.dropna(subset=["Score"])
# df = df.dropna(subset=["Episodes"])

In [12]:
df.shape

(14562, 28)

In [13]:
df.isna().sum()
## Only left 3 more features to work with their Nan's.

ID                     0
Title                  0
Synonyms               0
Japanese               0
English                0
Synopsis               0
Type                   0
Episodes              78
Status                 0
Start_Aired            0
End_Aired              0
Premiered              0
Broadcast              0
Producers              0
Licensors              0
Studios                0
Source                 0
Genres                 0
Themes                 0
Demographics           0
Duration_Minutes      14
Rating                 0
Score                  0
Scored_Users           0
Ranked              1461
Popularity             0
Members                0
Favorites              0
dtype: int64

In [14]:
### It is onlt 14 nan's on Duration_minutes so it won't ruin the dataset. Tho it is safe to drop them too

df['Duration_Minutes'] = df['Duration_Minutes'].fillna(value=1)

In [15]:
### Proceed to drop useless columns
df2 = df[["Duration_Minutes","Themes","Genres","Title","English","Rating","Source","Synopsis","Episodes","Premiered"]]

In [16]:
df2.head(3)

,Duration_Minutes,Themes,Genres,Title,English,Rating,Source,Synopsis,Episodes,Premiered
0,24.0,"Gore, Military, Survival","Action, Drama",Shingeki no Kyojin,Attack on Titan,R - 17+ (violence & profanity),Manga,"Centuries ago, mankind was slaughtered to near...",25.0,Spring 2013
1,23.0,Psychological,"Supernatural, Suspense",Death Note,Death Note,R - 17+ (violence & profanity),Manga,"Brutal murders, petty thefts, and senseless vi...",37.0,Fall 2006
2,24.0,Military,"Action, Adventure, Drama, Fantasy",Fullmetal Alchemist: Brotherhood,Fullmetal Alchemist Brotherhood,R - 17+ (violence & profanity),Manga,After a horrific alchemy experiment goes wrong...,64.0,Spring 2009


In [17]:
df2[df2.duplicated(['Title'])]
# No duplicated values

,Duration_Minutes,Themes,Genres,Title,English,Rating,Source,Synopsis,Episodes,Premiered


In [18]:
df2.isna().sum()
## Episodes only matters for an easier comprehension of the dataset

Duration_Minutes     0
Themes               0
Genres               0
Title                0
English              0
Rating               0
Source               0
Synopsis             0
Episodes            78
Premiered            0
dtype: int64

In [19]:
df2.isna().sum()

Duration_Minutes     0
Themes               0
Genres               0
Title                0
English              0
Rating               0
Source               0
Synopsis             0
Episodes            78
Premiered            0
dtype: int64

In [20]:
print(df2.loc[df2.Duration_Minutes == "None", "Duration_Minutes"])
print(df2.loc[df2.Themes == "None", "Themes"])
print(df2.loc[df2.Genres =="None", "Genres"])
print(df2.loc[df2.Title == "None", "Title"])
print(df2.loc[df2.English == "None", "English"])
print(df2.loc[df2.Rating == "None", "Rating"])

Series([], Name: Duration_Minutes, dtype: float64)
Series([], Name: Themes, dtype: object)
Series([], Name: Genres, dtype: object)
Series([], Name: Title, dtype: object)
Series([], Name: English, dtype: object)
2633     None
3604     None
4919     None
5787     None
5835     None
         ... 
14409    None
14456    None
14565    None
14711    None
14734    None
Name: Rating, Length: 97, dtype: object


C:\Users\CLIENTE\anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


##### is important to know that Duration in Minutes tells a lot about the "anime", "animes" with less than 10 minutes aren't animes or are anime msuic videos or commercials, some animes last 12 or 11 minutes.
So we quickly drop those before continuing

In [21]:
df2.loc[df2.Duration_Minutes < 10 ,"Duration_Minutes"].value_counts().sum()

3963

In [22]:
df2 = df2.loc[df2.Duration_Minutes > 10]
## 3963 to drop

# More data exploration and sieving

In [23]:
df2.shape

(10401, 10)

In [24]:
#df22 = df2.copy()

In [25]:
# df2 = df2.loc[df2.Episodes > 1]

In [26]:
#df2.Episodes.value_counts()

In [27]:
df2.head()

,Duration_Minutes,Themes,Genres,Title,English,Rating,Source,Synopsis,Episodes,Premiered
0,24.0,"Gore, Military, Survival","Action, Drama",Shingeki no Kyojin,Attack on Titan,R - 17+ (violence & profanity),Manga,"Centuries ago, mankind was slaughtered to near...",25.0,Spring 2013
1,23.0,Psychological,"Supernatural, Suspense",Death Note,Death Note,R - 17+ (violence & profanity),Manga,"Brutal murders, petty thefts, and senseless vi...",37.0,Fall 2006
2,24.0,Military,"Action, Adventure, Drama, Fantasy",Fullmetal Alchemist: Brotherhood,Fullmetal Alchemist Brotherhood,R - 17+ (violence & profanity),Manga,After a horrific alchemy experiment goes wrong...,64.0,Spring 2009
3,24.0,"Parody, Super Power","Action, Comedy",One Punch Man,One Punch Man,R - 17+ (violence & profanity),Web manga,The seemingly unimpressive Saitama has a rathe...,12.0,Fall 2015
4,23.0,"Love Polygon, Video Game","Action, Adventure, Fantasy, Romance",Sword Art Online,Sword Art Online,PG-13 - Teens 13 or older,Light novel,Ever since the release of the innovative Nerve...,25.0,Summer 2012


In [28]:
df2.tail(10)

,Duration_Minutes,Themes,Genres,Title,English,Rating,Source,Synopsis,Episodes,Premiered
16714,15.0,Music,Avant Garde,Sanuki Eiga-sai Opening Eizou,Unknown,G - All Ages,Original,Sanuki Film Festival in 2015 and 2016 had open...,2.0,Unknown
16768,38.0,Music,Unknown,Yanoya,Unknown,G - All Ages,Original,"In 2011, Japanese musician Akiko Yano collabor...",1.0,Unknown
16783,16.0,Music,Unknown,Wego Graphics,WEGO GRAPHICS,G - All Ages,Original,Commercials for the WEGO GRAPHICS brand.,2.0,Unknown
16812,35.0,Music,Unknown,Hakodate Musical,Hakodate Musical Opening Video,G - All Ages,Original,"WOW Inc.'s first architectural project: ""Futur...",1.0,Unknown
16889,30.0,"Mythology, Super Power","Action, Comedy, Supernatural",Shounen Sunday CM: Devil & Devil,Unknown,PG-13 - Teens 13 or older,Manga,Devil & Devil commercial for Shounen Sunday ma...,1.0,Unknown
16956,13.0,Unknown,"Avant Garde, Drama",Scripta Volant,Writings Fly Away,G - All Ages,Book,"Graduate animation by Ryo Orikasa, based The H...",1.0,Unknown
17000,30.0,Historical,Unknown,Hirooka Asako,Unknown,G - All Ages,Original,Commercials explaining the founder of Daido Se...,2.0,Unknown
17024,30.0,"Music, Racing",Unknown,Race Eizou-hen,Unknown,G - All Ages,Original,Advertisements for JRA-VAN's racehorse software.,2.0,Unknown
17059,15.0,Unknown,Avant Garde,Port of Wormy,Port of Wormy,G - All Ages,Original,A surreal short created by Mirai Mizue.,1.0,Unknown
17202,30.0,Unknown,"Comedy, Supernatural",Shounen Sunday CM: Tuxedo Gin,Unknown,PG-13 - Teens 13 or older,Manga,Tuxedo Gin commercial for Shounen Sunday magaz...,1.0,Unknown


In [29]:
### Seems like Premeired
### 3644 Unknowns in premeir are the same as 3644 NONE values in the SPLITTED PREMIER COL

In [30]:
print(df2.loc[df2.Premiered == "Unknown", "Premiered"].value_counts())

Unknown    6684
Name: Premiered, dtype: int64


In [31]:
df2.shape

(10401, 10)

In [32]:
df2["Synopsis"] = df2["Synopsis"].str.lower()
df2["Rating"] = df2["Rating"].str.lower()

In [33]:
df2 = df2[df2["Synopsis"].str.contains("commercial","commercials")==False]
df2 = df2[df2["Synopsis"].str.contains("commerials")==False]
df2 = df2[df2["Synopsis"].str.contains("music video","music videos")==False]
df2 = df2[df2["Synopsis"].str.contains("stop-motion","advertisements")==False]
df2 = df2[df2["Synopsis"].str.contains("advertisement")==False]
df2 = df2[df2["Rating"].str.contains("hentai")==False] 
### Just remember that Ratings for CHILDREN have popular shows like Pokemon, cardcaptor, digimon on it, you can keep
### them, after all it won't change the model too much
### df2 = df2[df2["Rating"].str.contains("children")==False]

In [34]:
df2.shape

(8889, 10)

In [35]:
## Drop the Unknown values based on Premiered, 
## after seeing that the Anime Titles in those rows are not animes or they have sexually explicit content so it is safe to drop 
## those rows

In [36]:
df2.loc[df2.Premiered == "Unknown", "Premiered"] = np.nan

In [37]:
df2.isna().sum()

Duration_Minutes       0
Themes                 0
Genres                 0
Title                  0
English                0
Rating                 0
Source                 0
Synopsis               0
Episodes              27
Premiered           5178
dtype: int64

In [38]:
df2 = df2.dropna(subset=["Premiered"])

In [39]:
df2.shape

(3711, 10)

In [40]:
L = ["Rating"]
for x in L:
    print(df2[x].value_counts())


pg-13 - teens 13 or older         2163
g - all ages                       501
r - 17+ (violence & profanity)     498
pg - children                      302
r+ - mild nudity                   228
none                                19
Name: Rating, dtype: int64


In [41]:
df44 = df2['Premiered'].str.split(' ', expand = True)

In [42]:

df44.rename(columns={ df44.columns[1]: "Year", df44.columns[0]: "Season"}, inplace = True)

data_types_dict = {'Year': float,
                                      }
df44 = df44.astype(data_types_dict)
df44 = df44[["Year"]]
df44 = df44.sort_values(by='Year',ascending=False)

In [43]:
df22 = pd.concat([df44,df2],axis = 1)

In [44]:
df22.dtypes

Year                float64
Duration_Minutes    float64
Themes               object
Genres               object
Title                object
English              object
Rating               object
Source               object
Synopsis             object
Episodes            float64
Premiered            object
dtype: object

In [45]:
print(df22.loc[df22.Year < 1970 , "Year"].value_counts().sum())

38


In [46]:
df22 = df22.loc[df22.Year > 1990 ]

In [47]:
df22.head()

,Year,Duration_Minutes,Themes,Genres,Title,English,Rating,Source,Synopsis,Episodes,Premiered
0,2013.0,24.0,"Gore, Military, Survival","Action, Drama",Shingeki no Kyojin,Attack on Titan,r - 17+ (violence & profanity),Manga,"centuries ago, mankind was slaughtered to near...",25.0,Spring 2013
1,2006.0,23.0,Psychological,"Supernatural, Suspense",Death Note,Death Note,r - 17+ (violence & profanity),Manga,"brutal murders, petty thefts, and senseless vi...",37.0,Fall 2006
2,2009.0,24.0,Military,"Action, Adventure, Drama, Fantasy",Fullmetal Alchemist: Brotherhood,Fullmetal Alchemist Brotherhood,r - 17+ (violence & profanity),Manga,after a horrific alchemy experiment goes wrong...,64.0,Spring 2009
3,2015.0,24.0,"Parody, Super Power","Action, Comedy",One Punch Man,One Punch Man,r - 17+ (violence & profanity),Web manga,the seemingly unimpressive saitama has a rathe...,12.0,Fall 2015
4,2012.0,23.0,"Love Polygon, Video Game","Action, Adventure, Fantasy, Romance",Sword Art Online,Sword Art Online,pg-13 - teens 13 or older,Light novel,ever since the release of the innovative nerve...,25.0,Summer 2012


In [48]:
df22.shape

(3241, 11)

In [49]:
df3 = df22.copy()

### Now we work with df3

In [50]:
df3.tail(3)

,Year,Duration_Minutes,Themes,Genres,Title,English,Rating,Source,Synopsis,Episodes,Premiered
15070,2014.0,23.0,Strategy Game,Fantasy,Huoli Shaonian Wang zhi Chuanqi Zaixian,Blazing Teens 5 Legendary Warriors,g - all ages,Original,the 3rd huoli shaonian wang animated season. i...,40.0,Fall 2014
15309,2010.0,11.0,Racing,Unknown,Kkoma Bus Tayo,Tayo the Little Bus Season 1,g - all ages,Original,tayo the little bus is about exciting adventur...,26.0,Summer 2010
15454,2013.0,12.0,Racing,Adventure,Chuldong! Super Wings,Super Wings,g - all ages,Original,super wings follows the adventures of an adora...,52.0,Fall 2013


In [51]:
### In Rating we have an issue with None values, we'll change the None by Unknown
df3.loc[df3.Rating == "None", "Rating"] = "Unknown"

In [52]:
print(df3.loc[df3.Rating == "None", "Rating"])

Series([], Name: Rating, dtype: object)


In [53]:
L = ["Genres","Themes","Source","Rating"]
for x in L:
    print(df3[x].value_counts())

Comedy                                       148
Comedy, Slice of Life                        105
Action, Sci-Fi                                99
Unknown                                       93
Comedy, Romance                               88
                                            ... 
Action, Comedy, Fantasy, Sci-Fi                1
Adventure, Comedy, Romance, Sci-Fi             1
Adventure, Fantasy, Mystery, Sci-Fi            1
Adventure, Comedy, Drama, Mystery, Sci-Fi      1
Drama, Fantasy, Supernatural, Suspense         1
Name: Genres, Length: 530, dtype: int64
Unknown                                               810
School                                                210
Mecha                                                 129
Historical                                             76
Super Power                                            76
                                                     ... 
Mecha, Military, Psychological, Space                   1
Isekai, Militar

In [54]:
df3.shape

(3241, 11)

In [55]:
# df2.loc[df2.Theme == "Unknown", "Theme"] = np.nan

##### Many Unknows so we have to apply Nan to them and then add proportionally some random Themes on the Nan values
######   
#### Then convert the remaining Nan in unknown and drop them with genre using .ILOC[] 

In [56]:
df3.loc[df3.Themes == "Unknown", "Themes"] = np.nan

In [57]:
df3.isna().sum()

Year                  0
Duration_Minutes      0
Themes              810
Genres                0
Title                 0
English               0
Rating                0
Source                0
Synopsis              0
Episodes             15
Premiered             0
dtype: int64

In [58]:
x1 = df3["Themes"].value_counts()

In [59]:
df3.shape

(3241, 11)

In [60]:
                  ##(SHAPE - THEME)

lista2 = []
for x in x1:
    lista2.append(x/(3038-734))   
    
lista3 = []
for x in lista2:
    lista3.append(x*3038)

In [61]:
#lista2

In [62]:
y = 0
for x in lista3:
    y = y + x
print(y)

3205.4592013888473


In [63]:
O_dfG = pd.DataFrame(df3["Themes"].value_counts())

values_genres = list(O_dfG.index)

len(values_genres)


451

In [64]:
len(lista3) == len(values_genres)

True

In [65]:
import math

In [66]:
# from sklearn.utils import shuffle
# df3=shuffle(df3)

# I shuffle the data for the next step, so we have Themes randomly and proportionally divided
## Tho it is really not necessary it wouldn't change much the dataset if we didnt' apply apply it.

In [67]:
for x, limite in zip(values_genres, lista3): 
    values = {"Themes": x}
    limite = limite * .01
    limite = math.ceil(limite)
    df3 = df3.fillna(value=values, limit=limite)

In [68]:
print(df3.isna().sum())

Year                  0
Duration_Minutes      0
Themes              354
Genres                0
Title                 0
English               0
Rating                0
Source                0
Synopsis              0
Episodes             15
Premiered             0
dtype: int64


In [69]:
values = {"Themes": "Unknown"}

df3 = df3.fillna(value=values)
df3["Themes"].value_counts()

Unknown                                           354
School                                            213
Mecha                                             131
Super Power                                        78
Historical                                         78
                                                 ... 
Mecha, School, Strategy Game                        2
Adult Cast, Gore, Organized Crime, Super Power      2
Gore, High Stakes Game, Survival                    2
Organized Crime, Super Power                        2
Mecha, Parody                                       2
Name: Themes, Length: 452, dtype: int64

In [70]:
## Now is time to drop Unknowns
## in order to drop them we need to convert Unknown to ZZZZ because is easier
df3['Themes'] = df3['Themes'].str.replace('Unknown', 'ZZZZ', regex=True).str.strip()

In [71]:
df3 = df3.sort_values(by=['Genres','Themes'],ascending=True)
df3.tail(12)

,Year,Duration_Minutes,Themes,Genres,Title,English,Rating,Source,Synopsis,Episodes,Premiered
8327,2017.0,23.0,Strategy Game,Unknown,Future Card Buddyfight Battsu,Future Card Buddyfight X,pg - children,Card game,the fourth season of future card buddyfight.,52.0,Spring 2017
8694,2008.0,24.0,Strategy Game,Unknown,Battle Spirits: Shounen Toppa Bashin,Unknown,pg - children,Card game,energetic and thrill-seeking sixth grader topp...,50.0,Fall 2008
8823,2012.0,12.0,Strategy Game,Unknown,Cross Fight B-Daman eS,B-Daman Fireblast,g - all ages,Unknown,"the anime takes place in crest land, where the...",52.0,Fall 2012
8200,2016.0,23.0,Strategy Game,Unknown,Future Card Buddyfight Triple D,Future Card Buddyfight Triple D,pg - children,Card game,third season of future card buddyfight.,51.0,Spring 2016
7372,2015.0,24.0,Strategy Game,Unknown,Future Card Buddyfight Hundred,Future Card Buddyfight Hundred,pg - children,Card game,while tasuku is off training and getting rest ...,50.0,Spring 2015
9160,2014.0,23.0,Strategy Game,Unknown,Hero Bank,Herobank,pg - children,Game,"in big money city, players participate in ""her...",51.0,Spring 2014
5885,2014.0,22.0,Strategy Game,Unknown,Majin Bone,Unknown,pg-13 - teens 13 or older,Game,"anime adaptation of majin bone, a digital card...",52.0,Spring 2014
4817,2022.0,23.0,ZZZZ,Unknown,Yurei Deco,Yurei Deco,pg-13 - teens 13 or older,Original,"in the cyber city tom sawyer, society revolves...",12.0,Summer 2022
6199,2011.0,24.0,ZZZZ,Unknown,Kaitou Tenshi Twin Angel: Kyun Kyun☆Tokimeki P...,Twin Angel Twinkle Paradise,pg-13 - teens 13 or older,Game,haruka minazuki and aoi kannazuki are two stud...,12.0,Summer 2011
12074,2007.0,11.0,ZZZZ,Unknown,Sugar Bunnies,Sugar Bunnies,g - all ages,Unknown,random cutesy edutainment tales aimed at small...,26.0,Spring 2007


In [72]:
## Keep in Mind that the amount of data with ZZZZ and Unknown will change because we are shuffling the data
df3.shape

(3241, 11)

In [73]:
3040-10

3030

In [74]:
df3 = df3.iloc[:3230,:]         ## Delete 79 with Unknown Values on Genres and Themes

In [75]:
df3.shape

(3230, 11)

In [76]:
df3.tail(20)

,Year,Duration_Minutes,Themes,Genres,Title,English,Rating,Source,Synopsis,Episodes,Premiered
15309,2010.0,11.0,Racing,Unknown,Kkoma Bus Tayo,Tayo the Little Bus Season 1,g - all ages,Original,tayo the little bus is about exciting adventur...,26.0,Summer 2010
8982,2010.0,25.0,"Racing, Space, Strategy Game",Unknown,Scan2Go,Scan2Go,g - all ages,Original,"sometime in the near future, in an age in whic...",52.0,Summer 2010
1376,2015.0,12.0,"Reverse Harem, School, Vampire",Unknown,"Diabolik Lovers More,Blood","Diabolik Lovers II More,Blood",r - 17+ (violence & profanity),Visual novel,"yui komori, still held captive by the sakamaki...",12.0,Fall 2015
618,2013.0,15.0,"Reverse Harem, School, Vampire",Unknown,Diabolik Lovers,Diabolik Lovers,r - 17+ (violence & profanity),Visual novel,"at the behest of her father, yui komori goes t...",12.0,Fall 2013
3826,2021.0,24.0,School,Unknown,Blue Reflection Ray,Blue Reflection Ray,pg-13 - teens 13 or older,Game,individual pieces of human emotion usually tak...,24.0,Spring 2021
12199,2001.0,20.0,School,Unknown,Go! Go! Itsutsugo Land,Unknown,g - all ages,Unknown,"the series is about five quintuplets, kabuto, ...",50.0,Spring 2001
12047,1998.0,24.0,School,Unknown,Himitsu no Akko-chan 3,Unknown,g - all ages,Unknown,"in the kagami household, the old western palm-...",44.0,Spring 1998
1651,2016.0,24.0,School,Unknown,Shoujo-tachi wa Kouya wo Mezasu,Girls Beyond the Wasteland,pg-13 - teens 13 or older,Visual novel,shoujo-tachi wa kouya wo mezasu is a series al...,12.0,Winter 2016
2193,2017.0,23.0,"School, Strategy Game",Unknown,Nana Maru San Batsu,Fastest Finger First,pg-13 - teens 13 or older,Manga,"as soon as he entered high school, the first y...",12.0,Summer 2017
3339,2015.0,23.0,Space,Unknown,Houkago no Pleiades (TV),Wish Upon the Pleiades,pg-13 - teens 13 or older,Original,the sky is the limit in houkago no pleiades. w...,12.0,Spring 2015


In [77]:
### Some Non-Animes are going to leak into the model, because there is now way to remove them unless it is manually. 
### Like Porong Porong Poror which is a CGI (clay like) show for kids with Duration of 10 minutes like a small quantity of animes. 


In [78]:
### LAST DATA CLEANING BEFORE CONVERTING IT INTO A CSV

df3['Rating'] = df3['Rating'].str.replace('[^a-zA-Z0-9]', ' ', regex=True).str.strip()
df3['Genres'] = df3['Genres'].str.replace('[^a-zA-Z0-9]', ' ', regex=True).str.strip()
df3['Source'] = df3['Source'].str.replace('[^a-zA-Z0-9]', ' ', regex=True).str.strip()
df3['Themes'] = df3['Themes'].str.replace('[^a-zA-Z0-9]', ' ', regex=True).str.strip()
#df3['Studio'] = df4['Studio'].str.replace('[^a-zA-Z0-9]', ' ', regex=True).str.strip()
df3['Title'] = df3['Title'].str.replace('[^a-zA-Z0-9]', ' ', regex=True).str.strip()
df3['Synopsis'] = df3['Synopsis'].str.replace('\d+', ' ', regex=True).str.strip()

df3['Rating'] = df3['Rating'].str.replace('\d+', '', regex=True).str.strip()
df3['Genres'] = df3['Genres'].str.replace('\d+', '', regex=True).str.strip()
df3['Source'] = df3['Source'].str.replace('\d+', '', regex=True).str.strip()
df3['Themes'] = df3['Themes'].str.replace('\d+', '', regex=True).str.strip()
#df4['Studio'] = df4['Studio'].str.replace('\d+', '', regex=True).str.strip()
df3['Title'] = df3['Title'].str.replace('\d+', '', regex=True).str.strip()
df3['Synopsis'] = df3['Synopsis'].str.replace('[^a-zA-Z0-9]', ' ', regex=True).str.strip()


In [79]:
df3.to_csv('AnimeDB4.csv')

In [80]:
# print(df3.loc[df3.Duration_Minutes < 5 , "Duration_Minutes"].value_counts().sum())

In [81]:
# df3.loc[df3.Duration_Minutes < 10 ]

In [82]:
# df3.loc[df3.Year == 1999 ]

In [83]:
# df3.Year.value_counts()

In [84]:
'''
L = ['Themes', 'Genres', 'Title', 'Rating',
       'Source', 'Synopsis', 'Episodes']
for x in L:
    print(df3[x].value_counts())
    '''

"\nL = ['Themes', 'Genres', 'Title', 'Rating',\n       'Source', 'Synopsis', 'Episodes']\nfor x in L:\n    print(df3[x].value_counts())\n    "

In [85]:
## Let's check the Source with Unknown V's
# df3.loc[df3.Source == "Unknown"] 

#### Another approach someone could take is to drop Rows based on the AGE the anime aired, usually old animes aren't popular and people prefer to watch new stuff or recent.  I did that in order to Build AnimeDB4.

#### Seems like this AnimeDB3 has little improvement compared to AnimeDB2 .

#### If you only want to build a recommendation sys for SHOWS (animes with episodes) and not movies you can try removing by Episodes. Just watch out the Nan values on Episodes, some of them are One Piece and Boruta (very popular animes)

#### Last, you can create AnimeDBp's with different criterias, a good criteria could be to create an AnimeDB just with more than 15 minutes Duration. But keep in mind that there are animes with 11 or 12 Duration_Minutes. 

#### So the best approach would be to create many AnimeDB's with many criterias, one for Adults, one for animes that last 15+ minutes, another one for short duration animes, other one only for Shows and not considering Ova's nor Movies. And running the model on each. 

#### AnimeDB4 is a DB that don't consider < 10 animes & Adult animes. So this DB can serve as a Base to create other DB's

In [86]:
#df[["Episodes","Genres","Themes","Title","Synopsis","Duration_Minutes"]].loc[df.Rating == "Rx - Hentai" ].tail(39)

In [87]:
#df.Rating.value_counts()

In [88]:
#df.loc[df.Rating == "R+ - Mild " ].head(30)

In [89]:
#df[["Episodes","Genres","Themes","Title","Synopsis","Duration_Minutes"]].loc[df.Rating == "PG - Children" ].head(39)